# Möbius Plateau **Auto‑Explorer v10** 🌌

**Grandma‑friendly version** – just paste your scan table in *Cell 2* and press **Runtime → Run all**.

What happens automatically:

1. Tags rows into plateaus at ε = 1 e‑4, 5 e‑5, 1 e‑5.  
2. Applies an extended op‑bank (×/÷ 9, 3, 6; φ, φ²; √φ; α, α², √α, α⁻¹; Feigenbaum δ and α; inverses; *1 − const*).  
3. Writes two Excel files: **plateaus_raw.xlsx** & **plateaus_ops.xlsx**.  
4. Prints the first line of every `(ε, op)` combo so you know it ran.

No widgets, no parameters – just run.

---

### Built‑in constants

| symbol | value | note |
|---|---|---|
| φ (golden ratio) | 1.618 033 988 75 | `phi` |
| φₘ (scan micro) | 0.003 942 ± 5 × 10⁻⁷ | `phi_micro` |
| α⁻¹ | 137.035 999 084 | fine‑structure |
| δ | 4.669 201 609 | Feigenbaum ratio |
| α_F | 2.502 907 875 | Feigenbaum α |

Feel free to edit **Cell 3** to add more ops.


In [1]:
import pandas as pd, numpy as np
from itertools import product


In [2]:

# ⬇️ Paste or replace your rows here ⬇️
raw_rows = [
    #  bits   k     φ-micro      λ      rel_err
    (64,  3.6, 0.003942, 0.0, 0.4117525875906678269),
    (64,  3.8, 0.003942, 0.0, 0.4117525875906678269),
    (64,  4.0, 0.003942, 0.0, 0.4117525875906678269),
    (64,  4.2, 0.003942, 0.0, 0.4117525875906678269),
    (128, 3.6, 0.003942, 0.0, 0.4117525875906678271),
    (128, 3.8, 0.003942, 0.0, 0.4117525875906678271),
    (128, 4.0, 0.003942, 0.0, 0.4117525875906678271),
    (128, 4.2, 0.003942, 0.0, 0.4117525875906678271),
]


df = pd.DataFrame(raw_rows, columns=['bits','k','phi','lam','rel_err'])
display(df.head())


,bits,k,phi,lam,rel_err
0,64,3.6,0.003942,0.0,0.411753
1,64,3.8,0.003942,0.0,0.411753
2,64,4.0,0.003942,0.0,0.411753
3,64,4.2,0.003942,0.0,0.411753
4,128,3.6,0.003942,0.0,0.411753


In [3]:

# === constants ===
phi      = 1.61803398875
phi_micro= 0.003942
alpha_inv= 137.035999084
alpha_fs = 1/alpha_inv
delta    = 4.669201609
alpha_F  = 2.502907875

# helper inverses
consts = {
    '9':9, '3':3, '6':6,
    'phi':phi, 'phi²':phi**2, '√phi':np.sqrt(phi),
    'alpha':alpha_fs, 'alpha²':alpha_fs**2, '√alpha':np.sqrt(alpha_fs), 'alpha⁻¹':alpha_inv,
    'delta':delta, 'delta²':delta**2, '√delta':np.sqrt(delta),
    'alpha_F':alpha_F, 'alpha_F²':alpha_F**2
}

OPS = {}
for name, c in consts.items():
    OPS[f'×{name}'] = lambda x, c=c: x*c
    OPS[f'÷{name}'] = lambda x, c=c: x/c
    OPS[f'1−{name}'] = lambda x, c=c: np.abs(1-c)   # const standalone


In [4]:

def tag_plateaus(frame: pd.DataFrame, eps: float):
    base = frame['rel_err'].min()
    out  = frame.copy()
    out['plateau'] = ((out['rel_err']-base)//eps).astype(int)
    return out

eps_grid = [1e-4,5e-5,1e-5]
views = []
for eps in eps_grid:
    tagged = tag_plateaus(df, eps)
    for op, fn in OPS.items():
        tmp = tagged.copy()
        tmp['rel_err'] = fn(tmp['rel_err'])
        tmp['ε'] = eps
        tmp['op'] = op
        views.append(tmp)
        print(f'ε={eps} op={op} → first row rel_err={tmp.rel_err.iloc[0]:.6g}')

out = pd.concat(views, ignore_index=True)
df.to_excel('plateaus_raw.xlsx', index=False)
out.to_excel('plateaus_ops.xlsx', index=False)
print('\nSaved plateaus_raw.xlsx and plateaus_ops.xlsx')


ε=0.0001 op=×9 → first row rel_err=3.70577
ε=0.0001 op=÷9 → first row rel_err=0.0457503
ε=0.0001 op=1−9 → first row rel_err=8
ε=0.0001 op=×3 → first row rel_err=1.23526
ε=0.0001 op=÷3 → first row rel_err=0.137251
ε=0.0001 op=1−3 → first row rel_err=2
ε=0.0001 op=×6 → first row rel_err=2.47052
ε=0.0001 op=÷6 → first row rel_err=0.0686254
ε=0.0001 op=1−6 → first row rel_err=5
ε=0.0001 op=×phi → first row rel_err=0.66623
ε=0.0001 op=÷phi → first row rel_err=0.254477
ε=0.0001 op=1−phi → first row rel_err=0.618034
ε=0.0001 op=×phi² → first row rel_err=1.07798
ε=0.0001 op=÷phi² → first row rel_err=0.157275
ε=0.0001 op=1−phi² → first row rel_err=1.61803
ε=0.0001 op=×√phi → first row rel_err=0.523757
ε=0.0001 op=÷√phi → first row rel_err=0.3237
ε=0.0001 op=1−√phi → first row rel_err=0.27202
ε=0.0001 op=×alpha → first row rel_err=0.0030047
ε=0.0001 op=÷alpha → first row rel_err=56.4249
ε=0.0001 op=1−alpha → first row rel_err=0.992703
ε=0.0001 op=×alpha² → first row rel_err=2.19264e-05
ε=0.0001 